### Control TPT pictet

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
from pathlib import Path
from TPT_generator_python import DataBucket, TPTFetcher, TPTGenerator

In [2]:
DATE = "2021-03-31"
CLIENT = "Pictet"
ISIN = "LU1787059465"
SYMADJ = -0.0
SOURCE_DIR = Path("../data/pictet_20210331")
OUTPUT_DIR = './production/control'
ALL = slice(None)

### Check generation inputs

In [3]:
generator = TPTGenerator(DATE,
                         CLIENT,
                         OUTPUT_DIR,
                         SOURCE_DIR,
                         sym_adj=SYMADJ,
                         shareclass_isin=ISIN)
generator


    Reporting date:          2021-03-31
    Client:                  Pictet
    Shareclass isin:         LU1787059465 
    number of instruments:   93
        

In [4]:
generator.data_bucket.get_instruments(ALL).shape
generator.data_bucket.get_groups()

0     20
11    28
14    56
Name: group_id, dtype: int64

### Identify shareclass in subfund

In [5]:
print("subfund currency: ", generator.data_bucket.get_subfund_infos('subfund_currency'))
generator.data_bucket.get_shareclass_infos(isin=ALL)

subfund currency:  USD


,id,shareclass,shareclass_currency,shareclass_name,id_subfund,type_tpt
code_isin,,,,,,
LU1787059465,463,A1E,EUR,Decalia SICAV - Circular Economy - A1E,148,1
LU1787059036,464,A1U,USD,Decalia SICAV - Circular Economy - A1U,148,1
LU1787060471,468,R,USD,Decalia SICAV - Circular Economy - R,148,1
LU1787061107,465,I,USD,Decalia SICAV - Circular Economy - I,148,1
LU1787061362,466,IC,CHF,Decalia SICAV - Circular Economy - IC,148,1
LU1787061529,467,IE,EUR,Decalia SICAV - Circular Economy - IE,148,1


### Control that all shareclasses are retrieved

In [6]:
navs = generator.data_bucket.get_shareclass_nav(isin=ALL, info=['shareclass_total_net_asset_sc_ccy',
                                                                'shareclass_total_net_asset_sf_ccy',
                                                                'subfund_total_net_asset'])
navs

,shareclass_total_net_asset_sc_ccy,shareclass_total_net_asset_sf_ccy,subfund_total_net_asset
shareclass,,,
LU1787059465,6551602.24,7700098.11,1.151455e+08
LU1787059036,3591420.28,3591420.28,1.151455e+08
LU1787060471,35515941.21,35515941.21,1.151455e+08
LU1787061107,29493345.31,29493345.31,1.151455e+08
LU1787061362,14499493.67,15406147.01,1.151455e+08
LU1787061529,19942609.58,23438549.04,1.151455e+08


In [7]:
instruments = generator.data_bucket.get_instruments_by_index(ALL)
instruments["market_value_fund"].sum()
instruments_TNA = instruments["market_value_fund"].sum()
instruments_TNA

115145510.69

In [8]:
print("sum of shareclass navs: ", navs["shareclass_total_net_asset_sf_ccy"].sum())
print("TNA of subfund:         ", navs["subfund_total_net_asset"].iloc[0])
## if sum == TNA ==> no navs missing

sum of shareclass navs:  115145500.96000001
TNA of subfund:          115145500.96


In [9]:
generator.data_bucket.processing_data = None
dist_vec = generator.data_bucket.get_distribution_vector()
print(dist_vec.sum())
## if no fake line, should be equel to shareclass nav
print(generator.data_bucket.get_shareclass_nav(info=['shareclass_total_net_asset_sf_ccy']).iloc[0])

7700098.759867836
7700098.11


###  Control that the distribution correctly distributed the instruments

In [10]:
instruments_sum = generator.data_bucket.processing_data.loc[(ALL, ALL), "distribution"].sum()
sum_1 = generator.data_bucket.processing_data.loc[(ALL, navs.index[0]), "distribution"].sum()
print("sum of distributed instruments: ", instruments_sum)
print("subfund TNA: ", navs["subfund_total_net_asset"].iloc[0])

print("sum of instruments distributed to the shareclass: ", sum_1)
print("shareclass navs: ", generator.data_bucket.get_shareclass_nav("shareclass_total_net_asset_sf_ccy"))

sum of distributed instruments:  115145510.68999998
subfund TNA:  115145500.96
sum of instruments distributed to the shareclass:  7700098.759867836
shareclass navs:  7700098.11


In [11]:
total_diff = 0
for isin in generator.data_bucket.get_shareclass_infos(isin=ALL).index:
    sum_isin = generator.data_bucket.processing_data.loc[(ALL, isin), "distribution"].sum()
    nav_isin = generator.data_bucket.get_shareclass_nav("shareclass_total_net_asset_sf_ccy", isin=isin)
    diff = nav_isin - sum_isin
    total_diff += diff
    print("computed nav: ", sum_isin)
    print("   given nav: ", nav_isin)
    print("difference: ", diff)
print("total diff:", round(total_diff, 5))

computed nav:  7700098.759867836
   given nav:  7700098.11
difference:  -0.6498678354546428
computed nav:  3591420.584114442
   given nav:  3591420.28
difference:  -0.3041144423186779
computed nav:  35515944.217320375
   given nav:  35515941.21
difference:  -3.007320374250412
computed nav:  29493347.806244075
   given nav:  29493345.31
difference:  -2.496244076639414
computed nav:  15406148.308321137
   given nav:  15406147.01
difference:  -1.2983211372047663
computed nav:  23438551.01413213
   given nav:  23438549.04
difference:  -1.9741321317851543
total diff: -9.73


In [16]:
for isin in generator.data_bucket.get_shareclass_infos(isin=ALL).index:
    sum_weights = generator.data_bucket.processing_data.loc[(ALL, isin), "valuation weight"].sum().round(5)
    print(sum_weights)

1.0
1.0
1.0
1.0
1.0
1.0


In [18]:
for isin in generator.data_bucket.get_instruments_by_index(ALL).index:
    sum_distribution = generator.data_bucket.processing_data.loc[(isin, ALL), "distribution weight"].sum().round(5)
print((sum_distribution==1.0).all())

True
